In [ ]:
#uncomment and run to install textblob
#import sys
#!{sys.executable} -m pip install textblob
#!{sys.executable} -m textblob.download_corpora

In [1]:
import pandas as pd
from textblob import TextBlob

### What is the most positive sentence, and the most negative sentence in the earning calls?

### Incorporate subjectivity and select objective sentiments (low subjectivity)

In [2]:
data = pd.read_csv('data/EC10.csv')

In [3]:
data.head()

,id,text,date,company,sector
0,32934,"Zoe's Kitchen, Inc. (NYSE:ZOES)\r\nQ2 2015 Ear...",2015-08-28 00:11:00,"Zoe's Kitchen, Inc.",Consumer Services
1,32905,"Autodesk, Inc. (NASDAQ:ADSK)\r\nQ2 2016 Earnin...",2015-08-28 00:07:00,"Autodesk, Inc.",Technology
2,32926,Smith & Wesson Holding Corp. (NASDAQ:SWHC)\r\n...,2015-08-27 23:25:00,Smith & Wesson Holding Corporation,Capital Goods
3,32930,"Ulta Salon, Cosmetics & Fragrance, Inc. (NASDA...",2015-08-27 23:09:00,"Ulta Salon, Cosmetics & Fragrance, Inc.",Consumer Services
4,32907,Splunk Inc. (NASDAQ:SPLK)\r\nQ2 2016 Earnings ...,2015-08-27 22:56:00,Splunk Inc.,Technology


In [70]:
def most_polarized(text, direction="positive", objectivity=False):
    blob = TextBlob(text)
    polarities = {
        sentence.raw:sentence.sentiment[0] if not objectivity else sentence.sentiment[0] * (1 - sentence.sentiment[1]) 
    for sentence in blob.sentences}
    if direction=="positive":
        most_polarized = pd.Series(polarities).idxmax()
    elif direction=="negative":
        most_polarized = pd.Series(polarities).idxmin()
    else:
        most_polarized = None
    return most_polarized

In [71]:
data['most positive'] = data['text'].apply(most_polarized)

In [72]:
data['most negative'] = data['text'].apply(
    lambda text:  most_polarized(text, direction="negative")
)

In [74]:
#data['most negative'].to_list()

In [75]:
data['most positive objective'] = data['text'].apply(
    lambda text:  most_polarized(text, direction="positive", objectivity=True)
)

In [76]:
data['most negative objective'] = data['text'].apply(
    lambda text:  most_polarized(text, direction="negative", objectivity=True)
)

In [78]:
data['most positive objective'].to_list()

["Also during today's call, we will discuss non-GAAP measures, which we believe can be useful in evaluating our performance.",
 'Good afternoon.',
 'We have what we believe the best margins in the industry, so why not leverage that.',
 'Now turning to Ultamate Rewards loyalty program, we continue to reward our best guests with a differentiated experience, driving increased engagement and satisfaction.',
 'We continue to improve the core Splunk enterprise and all the core platform elements so that it can be the very best indexing engine and data analytics device known to man.',
 "Tony D. Bartel - Chief Operating Officer\r\nIt's probably the best indicator we have.",
 'Our Board of Directors unanimously concluded that this arrangement is the best way to maximize value of our stockholders, providing them with immediate cash value for their investment.',
 'Rudy Sankovic\r\nThank you, Operator and good afternoon, everyone.',
 'Todd Noden\r\nGood afternoon, everyone.',
 'So I think its best 

In [80]:
data['most negative objective'].to_list()

['And then secondarily, any thoughts on small-order delivery?',
 'If I just compare the press releases over the past couple of quarters here, in Q4 you talked about the majority of the subscriber additions being maintenance subscriptions.',
 "And then the promotional activity – promotional discounting that you're doing, given the dramatic rise in NICS data, why continue the promotional activity?",
 'Or what has that looked like in the past?',
 'Splunk helped them failed customer transactions by 95%, increased customer satisfaction and doubled the revenue through their mobile point of sales systems, Sephora said, “Splunk has helped us to move from ambiguity to transparency.',
 "Rob and Tony will provide color on the video game business and the Tech Brands' metrics.",
 'During this past 20 years, we have seen the coming to dominance of CMOS as a technological basis for image sensors and the exploding of the imaging market.',
 'From a competitive standpoint, though you guys being forced t